# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
installs = pd.read_pickle("../../../../data/tp2/installs_tp2_formateado.pkl")

In [3]:
targets_competencia = pd.read_pickle("../../../../data/tp2/ref_hashes_target.pkl")

---

### De installs, me quedo con sólo datos que correspondan a la competencia

In [4]:
installs_competencia = installs.loc[installs["device_id"].isin(targets_competencia.index)]

---

# Cargo features generados hasta el momento

## Sets de entrenamiento

In [5]:
# Ventanas de tres dias
Xs = {}
try:
    print("Cargando features")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = pd.read_csv("../../features/ventana_{}_entrenar_installs.csv".format(ventana_nro), index_col=0)
except FileNotFoundError:
    print("No se encontraron los features, se generarán los dataframe requeridos")
    for ventana_nro in range(1,5):    
        Xs[ventana_nro] = installs.loc[installs["ventana_{}".format(ventana_nro)]]["device_id"] \
            .drop_duplicates().to_frame().set_index("device_id").copy()

Cargando features
No se encontraron los features, se generarán los dataframe requeridos


## Set que se usará para predecir

In [5]:
# Features para hacer las predicciones que se suben, corresponden a la ventana 7
Xs_predecir = {}

try:
    print("Cargando features usados para predecir")
    Xs_predecir[7] = pd.read_csv("../../features/predecir_installs.csv", index_col=0)
except FileNotFoundError:
    print("No se encontraron los features usados para predecir, se generará el dataframe requerido")
    Xs_predecir[7] = targets_competencia.copy()

Cargando features usados para predecir
No se encontraron los features usados para predecir, se generará el dataframe requerido


## Diccionario de features

In [6]:
features = {}
# Todos los datasets tienen la misma cantidad y nombre de features, tomo la ventana 1
columnas_ya_generadas = list(Xs[1].columns.values)

---

# Creación de features

## Cantidad de instalaciones dentro de la ventana

In [7]:
def cantidad_de_instalaciones(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"date" : "count"}).rename(columns={"date": "cantidad_instalaciones_installs"})

In [8]:
features["cantidad_de_instalaciones"] = (cantidad_de_instalaciones, ["cantidad_instalaciones_installs"])

---

## Tiempo desde la última instalacion hasta el fin de la ventana

In [9]:
def tiempo_desde_ult_instalacion_hasta_fin_ventana(dataframe, nro_ventana):
    return (np.datetime64("2019-04-2{}".format(nro_ventana)) - dataframe.groupby("device_id").agg({"date" : "max"})["date"]).dt.total_seconds().to_frame().rename(columns={"date": "tiempo_ultima_instalacion_hasta_fin_ventana"})

In [10]:
features["tiempo_desde_ult_instalacion_hasta_fin_ventana"] = (tiempo_desde_ult_instalacion_hasta_fin_ventana, ["tiempo_ultima_instalacion_hasta_fin_ventana"])

---

## Cantidad de aplicaciones distintas instaladas

In [11]:
def cantidad_aplicaciones_distintas_instaladas(dataframe, nro_ventana):
    return dataframe.groupby("device_id").agg({"application_id" : "nunique"}).rename(columns={"application_id": "cantidad_aplicaciones_diferentes_installs"})

In [12]:
features["cantidad_aplicaciones_distintas_instaladas"] = (cantidad_aplicaciones_distintas_instaladas, ["cantidad_aplicaciones_diferentes_installs"])

---

# Genero los features

## Función generadora de features en ventanas

In [13]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos, ventana_inicial, ventana_final):
    """El rango se toma como python, [ventana_inicial, ventana_final)"""
    for ventana_nro in range(ventana_inicial, ventana_final):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True, how="left")

### Genero los features

In [14]:
COLUMNAS_QUE_GENERA_EL_FEATURE = 1
FUNCION_PARA_CALCULAR_EL_FEATURE = 0

for nombre_feature in features:    
    columnas_generadas = features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE]
    feature_ya_generado = False
    for columna in columnas_generadas:
        if columna in columnas_ya_generadas:
            feature_ya_generado = True
            break
    
    if feature_ya_generado:
        continue

    generar_feature_en_ventanas(installs, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs, 1, 5)
    generar_feature_en_ventanas(installs_competencia, features[nombre_feature][FUNCION_PARA_CALCULAR_EL_FEATURE], Xs_predecir, 7, 8)    
    columnas_ya_generadas.extend(features[nombre_feature][COLUMNAS_QUE_GENERA_EL_FEATURE])

---

# Guardado de los sets de entrenamiento y el set de prediccion

In [16]:
for ventana_nro in range(1, 5):
    Xs[ventana_nro].to_csv("../../features/ventana_{}_entrenar_installs.csv".format(ventana_nro))

Xs_predecir[7].to_csv("../../features/predecir_installs.csv")

---